<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%bash
# apt-get update
# apt-get install g++ openjdk-8-jdk python-dev python3-dev
# pip3 install JPype1
# pip3 install konlpy

In [2]:
# %env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
# # Mecab 설치
# !apt-get update
# !apt-get install g++ openjdk-8-jdk 
# !pip3 install konlpy JPype1-py3
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [4]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

%matplotlib inline
# 한글 지원 폰트
plt.rc('font', family='NanumBarunGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False

import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from konlpy.tag import Mecab

vocab_size = 32000

data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_1000.csv')

In [6]:
data

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화
...,...,...,...,...,...
4908,3908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화
4909,3909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화
4910,3910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화
4911,3911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화


In [7]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면']

In [8]:
mecab = Mecab() 

data['tokenized'] = data['non_label_sentence'].apply(mecab.morphs)
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
data

,Unnamed: 0,non_label_sentence,label_sentence,class,binary_class,tokenized
0,0,아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려옴\...,1:아오 요즘 맨날 늦게 자고 일찍 일어나니까 피부가 꺼슬꺼슬 다크 서클 잔뜩 내려...,일반 대화,일반 대화,"[아오, 요즘, 맨날, 늦, 자, 일찍, 일어나, 니까, 피부, 꺼슬꺼슬, 다크, ..."
1,1,고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n우리집 고양이는 보일러 틀어주...,1:고양이를 키우는 건 정말 즐거운 일이야 감자를 캐자\n2:우리집 고양이는 보일러...,일반 대화,일반 대화,"[고양이, 키우, 건, 정말, 즐거운, 일, 야, 감자, 캐, 자, 우리, 집, 고..."
2,2,조금 전에 뉴스 본 거 있어\n아니 뉴스에 무슨 기사 떴어\n완전 조금 전에 대박 ...,1:조금 전에 뉴스 본 거 있어\n2:아니 뉴스에 무슨 기사 떴어\n1:완전 조금 ...,일반 대화,일반 대화,"[조금, 전, 뉴스, 본, 거, 있, 어, 아니, 뉴스, 무슨, 기사, 떴, 어, ..."
3,3,난 강아지 같은 푸근한 인상이 좋아\n오 나는 원래 고양이 상 좋아했는데 나도 강아...,1:난 강아지 같은 푸근한 인상이 좋아\n2:오 나는 원래 고양이 상 좋아했는데 나...,일반 대화,일반 대화,"[난, 강아지, 같, 푸근, 인상, 좋, 아, 오, 나, 원래, 고양이, 상, 좋,..."
4,4,니는 폰 게임 자주 하는 편이가\n가끔 삘 받으면 하긴 하는데 거의 안 하는 편 너...,1:니는 폰 게임 자주 하는 편이가\n2:가끔 삘 받으면 하긴 하는데 거의 안 하는...,일반 대화,일반 대화,"[니, 폰, 자주, 편이, 가끔, 삘, 받, 으면, 긴, 는데, 거의, 안, 편, ..."
...,...,...,...,...,...,...
4908,3908,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,1:준하야 넌 대가리가 왜이렇게 크냐\n2:내 머리가 뭐\n1:밥먹으면 대가리만 크...,기타 괴롭힘 대화,공격 대화,"[준, 야, 넌, 대가리, 왜, 이렇게, 크, 냐, 내, 머리, 뭐, 밥, 먹, 으..."
4909,3909,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,갈취 대화,공격 대화,"[내, 지금, 너, 아들, 김길준, 데리, 있, 어, 살리, 싶, 으면, 계좌, 1..."
4910,3910,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,직장 내 괴롭힘 대화,공격 대화,"[나, 씨, 같, 사람, 보, 참, 신기, 더라, 어떻게, 저렇게, 살, 왜, 그래..."
4911,3911,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,1:누구맘대로 여기서 장사하래\n2:이게 무슨일입니까\n1:남의 구역에서 장사하려면...,갈취 대화,공격 대화,"[누구, 맘대로, 여기, 서, 장사, 래, 이게, 무슨, 일, 입니까, 남, 구역,..."


In [9]:
conversation = data['tokenized'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(conversation)

In [10]:
len(tokenizer.word_index)

17536

In [11]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 17536
등장 빈도가 1번 이하인 희귀 단어의 수: 6836
단어 집합에서 희귀 단어의 비율: 38.98266423357664
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.470831872040209


In [12]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 10702


In [13]:
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(conversation)
X_data = tokenizer.texts_to_sequences(conversation)

In [14]:
padded = pad_sequences(X_data)
print(padded)

[[   0    0    0 ...  451    7  433]
 [   0    0    0 ...  223  530    7]
 [   0    0    0 ...    1 1441    1]
 ...
 [   0    0    0 ...    3   13  193]
 [   0    0    0 ...  207   11   17]
 [   0    0    0 ...  419   57 2098]]


In [15]:
np.shape(padded)

(4913, 485)

In [16]:
train_data = padded
train_label = data['class']

In [17]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화': 4}

In [18]:
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)
print(train_label)

      0  1  2  3  4
0     0  0  0  0  1
1     0  0  0  0  1
2     0  0  0  0  1
3     0  0  0  0  1
4     0  0  0  0  1
...  .. .. .. .. ..
4908  0  0  0  1  0
4909  0  1  0  0  0
4910  0  0  1  0  0
4911  0  1  0  0  0
4912  0  0  1  0  0

[4913 rows x 5 columns]


In [19]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

3930 491 492
3930 491 492


In [20]:
embedding_dim = 256
hidden_units = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout = 0.3)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout = 0.3)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units//2, dropout = 0.3)))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         2739712   
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        394240    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 256)        394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 10702)             1380558   
                                                        

In [30]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint(filepath='/content/sample_data/best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=64,
                    callbacks=[es, mc],
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
15/62 [======>.......................] - ETA: 7s - loss: 0.0897 - accuracy: 0.9781

KeyboardInterrupt: ignored

In [22]:
loaded_model = load_model('/content/sample_data/best_model.h5')
print("테스트 정확도: %.4f" % (loaded_model.evaluate(test_X, test_Y)[1]))

16/16 [==============================] - 3s 54ms/step


In [ ]:
pred = loaded_model.predict(test_X)

In [23]:
from sklearn.metrics import classification_report

pred_label = []
test_label = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.85      0.57      0.68        69
           1       0.69      0.80      0.74        94
           2       0.84      0.84      0.84       110
           3       0.75      0.69      0.71       115
           4       0.85      1.00      0.92       104

    accuracy                           0.79       492
   macro avg       0.80      0.78      0.78       492
weighted avg       0.79      0.79      0.79       492



In [24]:
test_X

array([[   0,    0,    0, ...,   61,   50,  351],
       [   0,    0,    0, ...,    8,   11,   17],
       [   0,    0,    0, ...,   23,   11,   17],
       ...,
       [   0,    0,    0, ..., 1457,  144,    4],
       [   0,    0,    0, ...,  299,   52,  117],
       [   0,    0,    0, ...,  258, 1689,  258]], dtype=int32)

In [37]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화': 4}
lidx2labels = {str(v) : k for k, v in labels.items()}

In [25]:
# 잘못 분류한 부분 출력
error_idx = []
error_label = []

for idx, val in enumerate(pred_label):
  if val != test_label[idx]:
    error_idx.append(idx)
    error_label.append(val)

In [26]:
len(error_idx)

103

In [27]:
error = test_X[error_idx]

In [28]:
error_text = tokenizer.sequences_to_texts(error)

In [35]:
lidx2labels[0]

'협박 대화'

In [36]:
error_label[0]

'0'

In [40]:
test_label[0]

'2'

In [39]:
result = []

for i in range(len(error_idx)):
  result.append([error_text[i], lidx2labels[error_label[i]], lidx2labels[test_label[idx]]])

result

[['너 이거 이렇게 라 했 어 안 했 어 저번 선배 님 설명 해 주 신 대로 했 는데요 장난 해 내 언제 이렇게 라 했 어 너 머리 장식 니 제 착각 했 나 봐요 죄송 합니다 죄송 야 정말 답답 해 죽 겠 죽 겠 어 죄송 합니다 죄송 할 짓 말 아야지 너 오늘 이거 기 전 까지 집 못 죄송 해요 좌 속 다는 말 밖에 못 너 앞 으로 실수 안 겠 습니다',
  '협박 대화',
  '일반 대화'],
 ['아줌마 손 든 거 뭐 예요 장바구니 인데요 아니 장바구니 아 는데 뭐 었 어요 오늘 애 먹일 기요 아줌마 미안 한데 나 집 애 둘 이나 있 는데 돈 없 어요 소고기 좀 같이 나눠 먹 요 아줌마 옷 잘 입 거 보 니 있 어 보이 는데 나 신랑 우리 버리 서 연락 안 돼서 돈 없 어요 주민 센터 같 은데 연락 해서 도움 좀 받 그래요 아 주말 주민 센터 여 거 봤 어요 우선 당장 굶 어 죽 생겼 으니까 좀 주 라고요 죄송 해요 저 볼게요 아줌마 그냥 나 아줌마 집 까지 쫓아가 서 그 집 앞 앉 아서 애 기다려요 기다려서 뭐 요 글쎄 뭐 할까 궁금 그냥 가시 든가 아니 그냥 소고기 좀 주 던가',
  '직장 내 괴롭힘 대화',
  '일반 대화'],
 ['뭐 라고 뭔데 뭐 요 너 나 누군지 몰라 몰라요 그래 너 잘 만났 이리 와봐 제 왜 요 그냥 얼굴 맘 안 어 오늘 기분 안 좋 은데 너 잘 걸렸 왜 그러 세요',
  '기타 괴롭힘 대화',
  '일반 대화'],
 ['왜 이렇게 시끄러워 그렇 다고 애 묶 어 둬요 다리 확 부러뜨려 버릴까 보 아니 아이 좀 뛰 었 다고 협박 거 야 못 할 거 같 아 내 가만히 있 거 같 아 애 항상 너 랑 있 건 아니 무슨 말 야 담 또 뛰 기 해 봐 진짜 부러뜨릴 거 니까 뭐 야 당신',
  '기타 괴롭힘 대화',
  '일반 대화'],
 ['야 너 그거 었 냐 뭐 아니 걔 있 잖아 이번 프로젝트 망쳐서 엄청 깨졌 다잖아 난 또 뭐 라고 걔 원래 일 못 기 로 유명 잖아 안녕 여기 있 었 어 아니 야 뭐 지금 뭐 거 야 너 말 야 너 이